In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()

from sklearn.datasets import make_moons
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.datasets import fetch_california_housing
from tensorflow.python.ops import math_ops

In [34]:
def train_input_fn(features,labels,batch_size):
    display('train_input_fn features ',type(features))
    display('train_input_fn labels',type(labels))
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
    return dataset.shuffle(1000,seed=42).repeat().batch(batch_size)


In [35]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    return dataset.batch(batch_size)
    

In [36]:
# create input function
housing = fetch_california_housing()
#display(housing.feature_names)

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
n,m = housing.data.shape
scaled_housing_data_bias = np.c_[np.ones((n,1)),scaled_housing_data]


df_housing = pd.DataFrame(scaled_housing_data_bias,columns=housing.feature_names + ['bias'])
#display(df_housing)
X = df_housing
y = housing.target.reshape(-1,1)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
display(X_train.shape,X_test.shape)

(16512, 9)

(4128, 9)

In [37]:
def my_model(features,labels,mode,params):
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
    
        """DNN with 2 hidden layers , and dropout of 0.1 probability"""
        input_layer = tf.feature_column.input_layer(features,
                                                   params['feature_columns'])

        hidden1 = tf.layers.dense(input_layer,units=params['n_hidden1'],
                                 activation=tf.nn.relu)
        dropout1 = tf.layers.dropout(inputs=hidden1,rate=0.1, 
                                     training= mode == tf.estimator.ModeKeys.TRAIN)
        hidden2 = tf.layers.dense(dropout1,units=params['n_hidden2'],
                                 activation=tf.nn.relu)
        dropout2 = tf.layers.dropout(inputs=hidden2,rate=0.1, 
                                     training= mode == tf.estimator.ModeKeys.TRAIN)
        
        #compute output
        output = tf.layers.dense(dropout2,params['n_output'])
        
        #compute predictions
        predicted_value = tf.cast(output,tf.float64)
        
        if mode == tf.estimator.ModeKeys.PREDICT:
            predictions = {
                'predicted_value' : predicted_value
            }

            return tf.estimator.EstimatorSpec(mode,predictions=predictions)
    
              
        
        #compute loss
        loss = tf.losses.mean_squared_error(labels=labels,
                                             predictions=predicted_value)
        


        rmse = tf.metrics.root_mean_squared_error(labels,predicted_value)
        metrics = {'rmse':rmse}
        #tf.summary.scalar('mse',accuracy[1])
       
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(mode,loss = loss,
                                             eval_metric_ops=metrics)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss,
                                     global_step = tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode,loss=loss,train_op=train_op)
    
    
    

In [40]:
config = tf.estimator.RunConfig(tf_random_seed=0)

feature_columns = []
print(type(X))
for key in df_housing.columns.values:
    feature_columns.append(tf.feature_column.numeric_column(key=key))
    
display(feature_columns)
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns':feature_columns,
        'n_hidden1': 10,
        'n_hidden2': 10,
        'n_output': 1
    },
    model_dir = 'my_model_2',
    config=config
)

<class 'pandas.core.frame.DataFrame'>


[NumericColumn(key='MedInc', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='HouseAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AveRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AveBedrms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='AveOccup', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Latitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Longitude', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='bias', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

INFO:tensorflow:Using config: {'_model_dir': 'my_model_2', '_tf_random_seed': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FC710B4FC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [41]:
batch_size = 100
train_step = 1000

classifier.train(
    input_fn=lambda:train_input_fn(X_train,y_train,batch_size),
    steps=train_step)


'train_input_fn features '

pandas.core.frame.DataFrame

'train_input_fn labels'

numpy.ndarray

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into my_model_2\model.ckpt.
INFO:tensorflow:loss = 8.699994, step = 0
INFO:tensorflow:global_step/sec: 390.137
INFO:tensorflow:loss = 0.6030164, step = 100 (0.257 sec)
INFO:tensorflow:global_step/sec: 742.723
INFO:tensorflow:loss = 0.40606594, step = 200 (0.135 sec)
INFO:tensorflow:global_step/sec: 711.12
INFO:tensorflow:loss = 0.71435744, step = 300 (0.141 sec)
INFO:tensorflow:global_step/sec: 795.608
INFO:tensorflow:loss = 0.44873896, step = 400 (0.126 sec)
INFO:tensorflow:global_step/sec: 754.04
INFO:tensorflow:loss = 0.45823926, step = 500 (0.133 sec)
INFO:tensorflow:global_step/sec: 726.571
INFO:tensorflow:loss = 0.39416528, step = 600 (0.138 sec)
INFO:tensorflow:global_step/sec: 748.269
INFO:tensorflow:loss

In [42]:
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(X_test,y_test,batch_size))

print('eval_result',eval_result)
print('eval_result',eval_result['rmse'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-09T15:22:53Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from my_model_2\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-09-15:22:53
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.4159876, rmse = 0.64614105
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: my_model_2\model.ckpt-1000
eval_result {'loss': 0.4159876, 'rmse': 0.64614105, 'global_step': 1000}
eval_result 0.64614105
